In [1]:
import os
import random
from sklearn.model_selection import train_test_split

In [2]:
# 定义将每个电极单独划分
electrode_regions = {'Fp1': ['Fp1'],
                     'Fp2': ['Fp2'],
                     'F3': ['F3'],
                     'F4': ["F4"],
                     'F7': ['F7'],
                     'F8': ['F8'],
                     'C3': ['C3'],
                     'C4': ['C4'],
                     'P3': ['P3'],
                     'P4': ['P4'],
                     'O1': ['O1'],
                     'O2': ['O2'],
                     'T3': ['T3'],
                     'T4': ['T4'],
                     'T5': ['T5'],
                     'T6': ['T6']}

In [3]:
# 设置文件夹路径和类别名称
folders = [r'D:\PycharmProjects\eegProject\data\Test_EEG\HC',
           r'D:\PycharmProjects\eegProject\data\Test_EEG\MDD',
           r'D:\PycharmProjects\eegProject\data\Test_EEG\BD']
class_names = ['HC', 'MDD', 'BD']

# 定义划分比例
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

# 创建存储数据集的字典
data_splits = {'train': [], 'val': [], 'test': []}

# 遍历每个类别的文件夹
for folder, class_name in zip(folders, class_names):
    # 获取文件夹中的Clean.mat文件列表
    file_list = os.listdir(folder)
    file_list = [os.path.join(folder, file) for file in file_list if file.endswith('Clean.mat')]

    # 随机划分数据
    train_files, remaining_files = train_test_split(file_list, train_size=train_ratio, random_state=42)
    val_files, test_files = train_test_split(remaining_files, test_size=test_ratio / (test_ratio + val_ratio),
                                             random_state=42)

    # 将数据加入到对应的数据集中
    data_splits['train'].extend([(file, class_name) for file in train_files])
    data_splits['val'].extend([(file, class_name) for file in val_files])
    data_splits['test'].extend([(file, class_name) for file in test_files])

# 打印每个划分的数据统计信息
for split, files in data_splits.items():
    print(f"{split} set: {len(files)} samples")

train set: 171 samples
val set: 57 samples
test set: 58 samples


In [4]:
def calculate_band_power(freqs, psd, freq_start, freq_end):
    # 确定频率范围索引
    freq_start_idx = np.argmax(freqs >= freq_start)
    freq_end_idx = np.argmax(freqs >= freq_end)

    # 提取频率范围内的功率谱密度
    selected_psd = psd[:, freq_start_idx:freq_end_idx+1]

    # 计算频带能量（求和）
    band_power = np.sum(selected_psd)  # 或使用 np.mean(selected_psd) 求平均值

    return band_power

In [5]:
label_dic = {'HC':0, 'MDD':1, "BD":2}

In [28]:
import numpy as np
from numpy import *
import mne
import scipy.io as sio
import csv

sub="D:\PycharmProjects\eegProject\data\Test_EEG\HC\Xdongjiangwen_EEG_ECClean.mat"
label = 'HC'

data = sio.loadmat(sub)
sample = data['EEG_ECClean']
eeg_data = sample["data"]
ch_names = sample['chanlocs'][0][0][0]['labels']
ch_names = [item[0] for item in ch_names]
info = mne.create_info(
    ch_names=['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6'],
    ch_types='eeg',
    sfreq=sample["srate"])
raw = mne.io.RawArray(eeg_data[0][0][:, 0:24000], info)
raw_res = raw.resample(200)

# 提取电极区域的时序信息和频域信息
region_data = {}

for i in range (10):  # 时序信息分为十个时间窗口
    for region, electrodes in electrode_regions.items():



        # 提取电极区域的数据
        region_eeg_data, _ = raw_res[electrodes, :]

        # 计算时序信息（均值）
        region_time_series = region_eeg_data.mean(axis=0)  # 根据需要进行修改

        region_time_series_list = []

        region_eeg_data_win = region_eeg_data[:, int(i/10*len(region_time_series)):int((i+1)/10*len(region_time_series))]
        region_time_series_win = region_eeg_data_win.mean(axis=0)

        for j in range(6):
            region_time_series_list.append(mean(region_time_series_win[int(j / 6 * len(region_time_series_win)):int((j + 1) / 6 * len(region_time_series_win))]))
        print(region_time_series_list)



Creating RawArray with float64 data, n_channels=16, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
[-0.8572266522049901, 0.8927008372545242, 0.554136288538575, 2.390927963294089, 5.368396179452539, 5.598363911993801]
[0.2731423847377302, -0.3310581029579042, -0.5883364743925629, -1.6917675795406097, -2.8230389898270367, -2.256756652966141]
[-0.11931232307106253, 0.13668333031237115, -0.4773643957450984, -0.7729462566110307, -0.34110973415896256, 0.1127251533046368]
[0.20802636670880026, -0.031132865305990087, 0.1374671215564014, -0.5355130435246975, -2.088810943886638, -2.86067296423018]
[-0.557211694046855, 0.4838058356568223, 0.4295898175612097, 2.1912520932033663, 4.2183735014870765, 3.3638227540068333]
[0.24003310531377786, 0.08170218750834429, 0.5660008011013268, 0.21527984060347063, -2.016955022979528, -3.875108389283996]
[0.10077546045184149, -0.018981971405446565, -0.7339699287153778, -1.1381707575730982, -0.8916462717205286, -0.16795512609183783]


In [24]:
region_eeg_data[:, int(1/10*len(region_time_series)):int((1+1)/10*len(region_time_series))]

array([[-2.04362954, -0.23082303, -0.05986227, ...,  0.43107433,
        -0.32606007, -0.40439462]])

In [25]:
region_eeg_data[:, int(2/10*len(region_time_series)):int((2+1)/10*len(region_time_series))]

array([[0.61431312, 2.93547238, 5.85854064, ..., 4.67027463, 4.77711593,
        4.62612093]])

In [14]:
region_time_series

array([-0.53245718, -2.70082246, -5.15269459, ...,  6.15968169,
        5.54785978,  4.02069462])

In [20]:
region_time_series[int(1/6*len(region_time_series)):int((1+1)/6*len(region_time_series))]

array([-0.73271121, -0.81014388, -2.38099469, ...,  1.41325785,
        1.05241775,  1.53088499])

In [12]:
import numpy as np
from numpy import *
import mne
import scipy.io as sio
import csv

sub="D:\PycharmProjects\eegProject\data\Test_EEG\HC\Xdongjiangwen_EEG_ECClean.mat"
label = 'HC'

data = sio.loadmat(sub)
sample = data['EEG_ECClean']
eeg_data = sample["data"]
ch_names = sample['chanlocs'][0][0][0]['labels']
ch_names = [item[0] for item in ch_names]
info = mne.create_info(
    ch_names=['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6'],
    ch_types='eeg',
    sfreq=sample["srate"])
raw = mne.io.RawArray(eeg_data[0][0][:, 0:24000], info)
raw_res = raw.resample(200)

# 提取电极区域的时序信息和频域信息
region_data = {}

for region, electrodes in electrode_regions.items():



    for i in range (10):  # 时序信息分为十个时间窗口

        # 提取电极区域的数据
        region_eeg_data, _ = raw_res[electrodes, :]

        # 计算时序信息（均值）
        region_time_series = region_eeg_data.mean(axis=0)  # 根据需要进行修改

        region_time_series_list = []

        region_eeg_data_win = region_eeg_data[:, int(i/10*len(region_time_series)):int((i+1)/10*len(region_time_series))]


        for j in range(6):
            region_time_series_list.append(mean(region_time_series[int(j/6*len(region_time_series)):int((j+1)/6*len(region_time_series))]))


        # 计算频域信息（功率谱密度,频带能量）
        info = mne.create_info(
            ch_names=electrodes,
            ch_types='eeg',
            sfreq=sample["srate"]
        )
        region_raw = mne.io.RawArray(region_eeg_data_win, info)

        region_psd, region_freqs = mne.time_frequency.psd_array_welch(region_raw.get_data(), sfreq=200)

        # 计算不同频段的频带能量，要有重叠
        # Delta波1-3Hz
        Delta = calculate_band_power(region_freqs, region_psd, 0, 4)
        # Theta波4-7Hz
        Theta = calculate_band_power(region_freqs, region_psd, 4, 7)
        # Alpha波9-13Hz
        Alpha = calculate_band_power(region_freqs, region_psd, 7, 13)
        # Beta波14-29Hz
        Beta = calculate_band_power(region_freqs, region_psd, 13, 30)
        # Gamma波30-48Hz
        Gamma = calculate_band_power(region_freqs, region_psd, 30, 50)

        # 存储时序信息和频域信息
        region_data = {'Delta_{}'.format(region): Delta,
                       'Theta_{}'.format(region): Theta,
                       'Alpha_{}'.format(region): Alpha,
                       'Beta_{}'.format(region): Beta,
                       'Gamma_{}'.format(region): Gamma,
                       'time_series_0_{}'.format(region): region_time_series_list[0],
                       'time_series_1_{}'.format(region): region_time_series_list[1],
                       'time_series_2_{}'.format(region): region_time_series_list[2],
                       'time_series_3_{}'.format(region): region_time_series_list[3],
                       'time_series_4_{}'.format(region): region_time_series_list[4],
                       'time_series_5_{}'.format(region): region_time_series_list[5],
                       }

Creating RawArray with float64 data, n_channels=16, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Ran

In [9]:
region_data

{'Fp1': {'Delta': 23.81750504784221,
  'Theta': 27.559117664658665,
  'Alpha': 33.10506392587306,
  'Beta': 13.17733671487598,
  'Gamma': 1.1267445029187966,
  'time_series_0': -0.4696125152427703,
  'time_series_1': -0.4469149373760445,
  'time_series_2': 0.17435673088021555,
  'time_series_3': 0.10508364689606273,
  'time_series_4': 0.1588315780134875,
  'time_series_5': 0.08454380658757855},
 'Fp2': {'Delta': 18.825261016137176,
  'Theta': 5.670756603959363,
  'Alpha': 21.92845357685406,
  'Beta': 12.956726173465452,
  'Gamma': 1.338814218424091,
  'time_series_0': -0.47989206119533606,
  'time_series_1': 1.9999003742123023,
  'time_series_2': -0.2506756227570585,
  'time_series_3': -0.22323509548883924,
  'time_series_4': -0.2412550686411559,
  'time_series_5': -0.2004263403397052},
 'F3': {'Delta': 7.243153218038496,
  'Theta': 5.6541653960393194,
  'Alpha': 19.796715684933634,
  'Beta': 8.888959764563285,
  'Gamma': 0.6651685931097889,
  'time_series_0': -0.05950215715053483,
  '

In [7]:
import numpy as np
from numpy import *
import mne
import scipy.io as sio
import csv

for sub, label in data_splits['val']:  # 被试循环

    data = sio.loadmat(sub)
    sample = data['EEG_ECClean']
    eeg_data = sample["data"]
    ch_names = sample['chanlocs'][0][0][0]['labels']
    ch_names = [item[0] for item in ch_names]
    info = mne.create_info(
        ch_names=['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6'],
        ch_types='eeg',
        sfreq=sample["srate"])
    raw = mne.io.RawArray(eeg_data[0][0][:, 0:24000], info)
    raw_res = raw.resample(200)

    # 提取电极区域的时序信息和频域信息
    region_data = {}

    for region, electrodes in electrode_regions.items():

        # 提取电极区域的数据
        region_eeg_data, _ = raw_res[electrodes, :]

        # 计算时序信息（均值）
        region_time_series = region_eeg_data.mean(axis=0)  # 根据需要进行修改

        for i in range (10):  # 时序信息分为十个时间窗口
            region_time_series_list = []

            for j in range(6):
                region_time_series_list.append(mean(region_time_series[int(j/6*len(region_time_series)):int((j+1)/6*len(region_time_series))]))

            region_eeg_data_win = region_eeg_data[:, int(i/10*len(region_time_series)):int((i+1)/10*len(region_time_series))]

            # 计算频域信息（功率谱密度,频带能量）
            info = mne.create_info(
                ch_names=electrodes,
                ch_types='eeg',
                sfreq=sample["srate"]
            )
            region_raw = mne.io.RawArray(region_eeg_data_win, info)

            region_psd, region_freqs = mne.time_frequency.psd_array_welch(region_raw.get_data(), sfreq=200)

            # 计算不同频段的频带能量，要有重叠
            # Delta波1-3Hz
            Delta = calculate_band_power(region_freqs, region_psd, 0, 4)
            # Theta波4-7Hz
            Theta = calculate_band_power(region_freqs, region_psd, 4, 7)
            # Alpha波9-13Hz
            Alpha = calculate_band_power(region_freqs, region_psd, 7, 13)
            # Beta波14-29Hz
            Beta = calculate_band_power(region_freqs, region_psd, 13, 30)
            # Gamma波30-48Hz
            Gamma = calculate_band_power(region_freqs, region_psd, 30, 50)

            # 存储时序信息和频域信息
            region_data[region] = {'Delta': Delta,
                                   'Theta': Theta,
                                   'Alpha': Alpha,
                                   'Beta': Beta,
                                   'Gamma': Gamma,
                                   'time_series_0': region_time_series_list[0],
                                   'time_series_1': region_time_series_list[1],
                                   'time_series_2': region_time_series_list[2],
                                   'time_series_3': region_time_series_list[3],
                                   'time_series_4': region_time_series_list[4],
                                   'time_series_5': region_time_series_list[5],
                                   }

            # 定义保存文件的路径
            save_path = r'D:\PycharmProjects\eegProject\data\extract_data\val\{}_region_data_{}_{}.csv'.format(label, sub[48:-16], i)

            # 打开文件进行写入
            with open(save_path, 'w', newline='') as file:
                writer = csv.writer(file)

                # 写入表头
                writer.writerow(['Region', 'Time_Series_0', 'Time_Series_1', 'Time_Series_2', 'Time_Series_3', 'Time_Series_4', 'Time_Series_5', 'Delta', 'Theta', 'Alpha', 'Beta', 'Gamma', 'Label'])

                # 遍历每个区域并写入数据
                for region, data in region_data.items():
                    region_time_series_0 = data['time_series_0']
                    region_time_series_1 = data['time_series_1']
                    region_time_series_2 = data['time_series_2']
                    region_time_series_3 = data['time_series_3']
                    region_time_series_4 = data['time_series_4']
                    region_time_series_5 = data['time_series_5']
                    region_Delta = data['Delta']
                    region_Theta = data['Theta']
                    region_Alpha = data['Alpha']
                    region_Beta = data['Beta']
                    region_Gamma = data['Gamma']

                    # 写入每行数据
                    writer.writerow(
                        [region + '_' + sub[48:-16], region_time_series_0, region_time_series_1, region_time_series_2, region_time_series_3, region_time_series_4, region_time_series_5, region_Delta, region_Theta, region_Alpha, region_Beta, region_Gamma, label_dic[label]])

Creating RawArray with float64 data, n_channels=16, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Ran

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_c

...

Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creati

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_c

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=1, n_times=2400
    Range : 0 ... 2399 =      0.000 ...    11.995 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_c